In [1]:
import keras
from keras.datasets import imdb
# num_words uses the x many most common words in the vocabulary (or the word index)
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=10000)

/Users/garethjones/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:955: UserWarning: Illegal line #3
	""""
"
	in file "/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)
/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:955: UserWarning: Illegal line #7
	""""
"
	in file "/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)

Bad key "Created on Thu May 24 09" on line 4 in
/Users/garethjones/anaconda3/lib/python3.6/site-packages/matplotl

In [2]:
import numpy as np

# do some things here to understand what's the shape, what are the labels etc?
# the max length of a neural network is 25000
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

# let's find out the min, max, avg length of each sample in the dataset. My way to do this would be:
a = []
for i in train_data:
    temp = len(i)
    a.append(temp)
print(max(a))
print(min(a))
print(np.mean(a))

# a more concise way to write this method would be the following using a list comprehension

b = [len(i) for i in train_data]
print(max(b))
print(min(b))
print(np.mean(b))

(25000,)
(25000,)
(25000,)
(25000,)
2494
11
238.71364
2494
11
238.71364


In [3]:
# let's work out what the hell is written here. The person who did some pre-processing kindly has already made a word index for us
# the word index shows each word and it's index position based on how common it is
word_index = imdb.get_word_index()
print(len(word_index))

# this loop goes through the items which are the key:value pairs using items()
# we then reverse the value / key position
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

# now we have a word index, we can decode our dataset into strings
review = train_data[0]
decoded_review = " ".join(
    [str(index) for index in review]
)
print(decoded_review)

# .get() goes to the index and says if the number given exists, print the index, if the number isn't there, then print ?
# we have to -3 from the index because there are normally 3 reserved words in NLP
decoded_review = " ".join(
    [reverse_word_index.get(index-3,'?') for index in review]
)
print(decoded_review)

88584
1 14 22 16 43 530 973 1622 1385 65 458 4468 66 3941 4 173 36 256 5 25 100 43 838 112 50 670 2 9 35 480 284 5 150 4 172 112 167 2 336 385 39 4 172 4536 1111 17 546 38 13 447 4 192 50 16 6 147 2025 19 14 22 4 1920 4613 469 4 22 71 87 12 16 43 530 38 76 15 13 1247 4 22 17 515 17 12 16 626 18 2 5 62 386 12 8 316 8 106 5 4 2223 5244 16 480 66 3785 33 4 130 12 16 38 619 5 25 124 51 36 135 48 25 1415 33 6 22 12 215 28 77 52 5 14 407 16 82 2 8 4 107 117 5952 15 256 4 2 7 3766 5 723 36 71 43 530 476 26 400 317 46 7 4 2 1029 13 104 88 4 381 15 297 98 32 2071 56 26 141 6 194 7486 18 4 226 22 21 134 476 26 480 5 144 30 5535 18 51 36 28 224 92 25 104 4 226 65 16 38 1334 88 12 16 283 5 16 4472 113 103 32 15 16 5345 19 178 32
? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself s

In [4]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence] = 1.0 # this is 2D indexing called fancy indexing. we put to a float because it's built on tensorflow which inputs floats
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
print(x_train[0])
print(x_train.shape)

[0. 1. 1. ... 0. 0. 0.]
(25000, 10000)


In [6]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")
print(y_train.shape)
print(y_test.shape)

(25000,)
(25000,)


In [ ]:
from keras import models, layers
model = models.Sequential()
# input shape is 10,000 because we turned each sample review into a one-hot encoded matrix
# of width 10,000. It's therefore is the number of features we provide the network in one go
model.add(layers.Dense(16,activation="relu",input_shape=(10000,)))
model.add(layers.Dense(16,activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

# we can use this approach to see the total list of optimizers / loss functions / metrics available etc
# from keras import optimizers, losses, metrics
# model.compile(optimizer=optimizers.RMSprop(lr=0.001),
#              loss=losses.binary_crossentropy(),
#              metrics=[metrics.binary_accuracy])

# we use cross entropy because this is a probability distribution. but we use a binary one in this case
model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
# let's create a validation dataset that's a small slice of the training data
# we have 25000 samples, so let's choose a subset of that
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

# then we do the same for our labels
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
# let's put the data into the model
history = model.fit(partial_x_train,partial_y_train,
          epochs=20,
          batch_size=512,
          validation_data=(x_val,y_val))

In [ ]:
import matplotlib.pyplot as plt

r = range(1, len(history.history['acc'])+1)

plt.plot(r, history.history["acc"], label = "Accuracy")
plt.plot(r,history.history["val_acc"],label="Validation Accuracy")
plt.legend()
plt.show()
plt.close()

plt.plot(r,history.history["loss"],label="Loss")
plt.plot(r,history.history["val_loss"],label="Validation Loss")
plt.legend()
plt.show()
plt.close()
